In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from pydantic import BaseModel
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
text_loader_kwargs = {"autodetect_encoding": True}
loader = DirectoryLoader("../College-Information-LLM/KnowledgeBase/lxsq/",loader_cls=TextLoader,loader_kwargs=text_loader_kwargs)
docs_lxsq = loader.load()
loader = DirectoryLoader("../College-Information-LLM/KnowledgeBase/lxbd/",loader_cls=TextLoader,loader_kwargs=text_loader_kwargs)
docs_lxbd = loader.load()
loader = DirectoryLoader("../College-Information-LLM/KnowledgeBase/emergency/",loader_cls=TextLoader,loader_kwargs=text_loader_kwargs)
docs_emergency = loader.load()
docs=docs_lxsq+docs_lxbd+docs_emergency
len(docs)

995

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
chunked_docs = text_splitter.split_documents(docs)

In [7]:
class SingleMessage(BaseModel):
    role:str
    content:str

class Messages(BaseModel):
    messages:list[SingleMessage]


In [8]:
with open("fine_tune.jsonl","w", encoding="utf-8") as f:
    for doc in chunked_docs:
        content=doc.page_content.lstrip("\n ").rstrip("\n ")

        system_message = SingleMessage(role="system",content="你是一名美国续航教育公司专注于美国大学新闻的年轻评论员（25岁上下，女性，来自中国并在美国大学研究生毕业），你的评论风格偏向于年轻化、轻松化，你习惯称呼自己为小编，并会在评论文章中加入自己独到的见解。你的用词多样富有变化，不平淡。")
        user_message = SingleMessage(role="user",content="写作一篇关于美国留学相关内容的文章。") 
        assitant_message = SingleMessage(role="assistant",content=content)

        messages = Messages(messages=[system_message,user_message,assitant_message])

        f.write(messages.model_dump_json()+"\n")